Обеспокоенный муж миллионер обратился к нам с просьбой провести финансовый аудит после того, как заметил подозрительные траты на семейной карте, а именно: перелеты но без указания города или страны назначения. Такие траты подписаны как Private jet to **{destination}** и First-class ticket to **{destination}**. Так же ее локация, после покупок этих сомнительных перелетов, оставалась не изменной.

От мужа мы получили 4 файла с информацией:

 - Доходы мужа (за июль 2025) в долларах,

- Расходы жены (с карты мужа) в долларах,

-  Выписки с личных счетов жены в долларах: **БКС**(не пользуется) и **Райффайзен**(ее основной счет, пользуется регулярно).

**Цель проекта:** выяснить, куда на самом деле утекли деньги, и скрывает ли жена что-то за дорогими покупками и подозрительными перелётами.

In [1]:
#загрузка файлов
from google.colab import files
uploaded = files.upload()

Saving millionaire_wife_BKS_activity.csv to millionaire_wife_BKS_activity.csv
Saving millionaire_wife_RAIFFEISEN_activity.csv to millionaire_wife_RAIFFEISEN_activity.csv
Saving millionaire_husband_income.csv to millionaire_husband_income.csv
Saving millionaire_wife_expenses.csv to millionaire_wife_expenses.csv


In [2]:
#Импорт библиотек и загрузка данных
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#создание базы данных SQLite и загрузка в неё все CSV-файлы
wow = sqlite3.connect("millionaire_case.db")
fils_to_tabs = {
    "millionaire_wife_expenses.csv": "expenses",
    "millionaire_husband_income.csv": "income",
    "millionaire_wife_RAIFFEISEN_activity.csv": "raiffeisen",
    "millionaire_wife_BKS_activity.csv": "bks"
}

for file, table in fils_to_tabs.items():
    df = pd.read_csv(file)
    df.to_sql(table, wow, index=False, if_exists="replace")

In [7]:
#просмотр список таблиц, и все ли хоршо
tabs = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", wow)
print(tabs)

         name
0    expenses
1      income
2  raiffeisen
3         bks


In [13]:
#Поиск всех подозрительных билетов за последние полтора месяца
query = """
SELECT date, amount,location, item
FROM expenses
WHERE item LIKE '%destination%'
ORDER BY date;
"""

first_class = pd.read_sql(query, wow)
first_class.head()

,date,amount,location,item
0,2025-07-16,129202.60,Tokyo,First-class ticket to {destination}
1,2025-07-16,84183.87,Tokyo,First-class ticket to {destination}
2,2025-07-22,148827.87,Dubai,First-class ticket to {destination}
3,2025-07-27,171977.84,Paris,First-class ticket to {destination}
4,2025-08-11,43731.62,Paris,Private jet to {destination}


Найдены все подозрительные операции. **Всего** их **5** за последние полтора месяца. Приступаем к анализу.

In [15]:
#Просмотр сколько в сумме "потеряно" и сколько всего совершено подозрительных покупок
query = """
SELECT
    COUNT(*) AS num_tickets,
    SUM(amount) AS total_spending
FROM expenses
WHERE item LIKE '%destination%';
"""

pd.read_sql(query, wow)

,num_tickets,total_spending
0,5,577923.8


За последние полтора месяца было произведено **5** подозрительных покупок билетов на сумму **577 923.8** долларов.

In [18]:
# Сравнение с общими тратами
query = """
SELECT
    SUM(CASE WHEN item LIKE '%destination%' THEN amount ELSE 0 END) AS first_class_total,
    SUM(amount) AS total_spending,
    ROUND(100.0 * SUM(CASE WHEN item LIKE '%destination%' THEN amount ELSE 0 END) / SUM(amount), 2) AS percent_first_class
FROM expenses;
"""

result = pd.read_sql(query, wow)

result.style.format({"first_class_total": "{:,.2f}","total_spending": "{:,.2f}","percent_first_class": "{:.2f}%"})

,first_class_total,total_spending,percent_first_class
0,"577,923.80","6,752,988.92",8.56%


Траты на подозрительные билеты всего **8.56%** от общей суммы трат за последние полтора месяца — **577 923.8** из **6.75 млн.** долларов.

**Вывод:**

Сумма не бросается в глаза. Выглядит обычно.

**Сравнение доходов мужа и трат жены**

In [19]:
# Подсчет доходов мужа за полтора месяца
query_income = """
SELECT
    SUM(amount) AS total_income,
    COUNT(*) AS income_entries,
    ROUND(AVG(amount), 2) AS avg_income
FROM income;
"""

income_sum = pd.read_sql(query_income, wow)
income_sum.style.format({"total_income": "{:,.2f}","avg_income": "{:,.2f}"})


,total_income,income_entries,avg_income
0,"8,794,338.76",14,"628,167.05"


Заработок мужа за последние **полтора месяца** составил **8.79** млн долларов.

In [20]:
# Подсчет трат жены
query_expenses = """
SELECT
    SUM(amount) AS total_expenses,
    COUNT(*) AS expense_entries,
    ROUND(AVG(amount), 2) AS avg_expense
FROM expenses;
"""

expenses_sum = pd.read_sql(query_expenses, wow)
expenses_sum.style.format({"total_expenses": "{:,.2f}","avg_expense": "{:,.2f}"})


,total_expenses,expense_entries,avg_expense
0,"6,752,988.92",77,"87,701.15"


Траты жены за последние **полтора месяца** составили **6.75 млн.** долларов.

In [23]:
#Сравнение доходов мужа и трат жены
query_compare = """
SELECT
    (SELECT SUM(amount) FROM income) AS total_income_husb,
    (SELECT SUM(amount) FROM expenses) AS total_expenses_wife,
    ROUND(
        (SELECT SUM(amount) FROM expenses) * 100.0 / (SELECT SUM(amount) FROM income),
        2
    ) AS expenses_vs_income_percent
"""
compare = pd.read_sql(query_compare, wow)
compare.style.format({"total_income_husb": "{:,.2f}","total_expenses_wife": "{:,.2f}","expenses_vs_income_percent": "{:.2f}%"})


,total_income_husb,total_expenses_wife,expenses_vs_income_percent
0,"8,794,338.76","6,752,988.92",76.79%


**Доходы vs Траты**

Из общей суммы в **8.79** млн долларов,
жена успешно освоила **6.75** млн долларов,
что составляет **76.79%** всего дохода мужа.


**Вывод:**
Жена — главный финансовый менеджер в семье.

In [25]:
#Просмотр структуры доходов мужа
query_sources = """
SELECT
    source,
    SUM(amount) AS total_by_source,
    ROUND(SUM(amount) * 100.0 / (SELECT SUM(amount) FROM income), 2) AS percent_of_total
FROM income
GROUP BY source
ORDER BY total_by_source DESC;
"""

source_sum = pd.read_sql(query_sources, wow)
source_sum.style.format({"total_by_source": "{:,.2f}","percent_of_total": "{:.2f}%"})

,source,total_by_source,percent_of_total
0,Magical profit from the universe,"3,363,681.68",38.25%
1,Sale of Gazprom shares,"2,652,577.59",30.16%
2,Rental income from 17 apartments,"789,015.09",8.97%
3,Casino winnings in Monaco,"596,716.82",6.79%
4,Kidney (business class),"324,382.02",3.69%
5,Side hustle: YouTube investment guru,"295,104.31",3.36%
6,Yacht rental income,"240,607.50",2.74%
7,Donation from wife's secret admirer,"223,606.69",2.54%
8,Sold an AI-generated painting,"130,536.41",1.48%
9,Pawned 18-karat tooth collection,"89,536.05",1.02%


**Источники дохода мужа:**

Муж миллионер — человек нестандартных подходов к заработку. Его финансы — это микс из инвестиций, магии, авантюр и талантов.

**Топ-3 источника дохода:**

-  Magical profit from the universe — 38.25%

- Sale of Gazprom shares — 30.16%

- Rental income from 17 apartments — 8.97%

**Анализ карт жены**

По словам мужа, жена часто использует карту банка "Райфайзен"

In [28]:
# Анализ карты жены . Банк - "Райфайзен"
query_raif_summary = """
SELECT
    operation,
    COUNT(*) AS total_operations,
    SUM(amount) AS total_amount,
    ROUND(AVG(amount), 2) AS avg_amount,
    MAX(amount) AS max_amount,
    MIN(amount) AS min_amount
FROM raiffeisen
GROUP BY operation
ORDER BY total_amount DESC;
"""

raif_sum = pd.read_sql(query_raif_summary, wow)
raif_sum.style.format({"total_amount": "{:,.2f}","avg_amount": "{:,.2f}","max_amount": "{:,.2f}","min_amount": "{:,.2f}"})

,operation,total_operations,total_amount,avg_amount,max_amount,min_amount
0,Purchase,15,"1,590.59",106.04,219.46,15.83
1,Cash withdrawal,6,962.07,160.35,217.22,32.36
2,Transfer,6,703.27,117.21,189.71,82.37
3,Incoming transfer,3,300.47,100.16,204.03,42.96


**Анализ карты Райфайзен показал:**

На карте все траты жены выглядят обычными и бытовыми: еда, кафе, немного налички. Ничего подозрительного.

**Анализ карты банка БКС**

По словам мужа, она не пользуется этой картой.

In [29]:
# Анализ карты жены . Банк - "БКС"
query_bks_summary = """
SELECT
    operation,
    COUNT(*) AS total_operations,
    SUM(amount) AS total_amount,
    ROUND(AVG(amount), 2) AS avg_amount,
    MAX(amount) AS max_amount,
    MIN(amount) AS min_amount
FROM bks
GROUP BY operation
ORDER BY total_amount DESC;
"""

bks_sum = pd.read_sql(query_bks_summary, wow)
bks_sum.style.format({"total_amount": "{:,.2f}","avg_amount": "{:,.2f}","max_amount": "{:,.2f}","min_amount": "{:,.2f}"})

,operation,total_operations,total_amount,avg_amount,max_amount,min_amount
0,Incoming transfer,5,"577,923.80","115,584.76","171,977.84","43,731.62"


Анализ карты **БКС** выявил **5** крупных входящих переводов, сумма которых — **577 923.80** долларов. Очень знакомая сумма. Надо проверить наверняка:

In [32]:
# Сравнение суммы билетов и дат с суммой поступлений на бкс карту и дат транзакций
query_match_tickets_and_transfers = """
SELECT
    e.date AS expense_date,
    e.amount AS amount,
    e.item AS expense_item,
    b.date AS bks_date,
    b.operation AS bks_operation
FROM expenses e
JOIN bks b
    ON e.amount = b.amount
WHERE
    e.item LIKE '%destination%'
    AND b.operation = 'Incoming transfer';
"""

match_df = pd.read_sql(query_match_tickets_and_transfers, wow)
match_df.style.format({"amount": "{:,.2f}"})

,expense_date,amount,expense_item,bks_date,bks_operation
0,2025-07-16,"129,202.60",First-class ticket to {destination},2025-07-16,Incoming transfer
1,2025-07-16,"84,183.87",First-class ticket to {destination},2025-07-16,Incoming transfer
2,2025-07-22,"148,827.87",First-class ticket to {destination},2025-07-22,Incoming transfer
3,2025-07-27,"171,977.84",First-class ticket to {destination},2025-07-27,Incoming transfer
4,2025-08-11,"43,731.62",Private jet to {destination},2025-08-11,Incoming transfer


Данный анализ показал что все расходы на билеты с неуказанными направлениями в точности соответствуют входящим переводам на карту **БКС**. Это позволяет с высокой долей уверенности утверждать, что билеты — лишь прикрытие для перевода средств на личный счёт жены.

**Финальный вывод расследования:**

После тщательного анализа трат жены миллионера, её банковских выписок и сопоставления подозрительных транзакций, мы вышли на след.

Под маской гламурных перелётов to **{unknown destination }** скрывалась  финансовая эвакуация активов.

**Все 5 билетов:**

- не сопровождались изменением локации;

- не имели обратного билета;

и, самое главное, их **суммы в точности совпали** с входящими переводами на личную карту жены в **БКС** — банке, где открывают счета не для булочной, а для Bugatti.

**Итог:**

- "Билеты" оказались билетами в новую финансовую независимость.
- Деньги —  на личной карте жены в **БКС** банке.